# SentEval usage example

* Clone repo from FAIR github
```
    git clone https://github.com/facebookresearch/SentEval.git
    cd SentEval/
```
* Dependencies:
    * Python 2/3 with NumPy/SciPy
    * Pytorch
    * scikit-learn>=0.18.0

* Install senteval
```
    python setup.py install
```
* Download datasets (it takes some time...)
    * these are downstream tasks
    * new Senteval also has probing tasks (https://github.com/facebookresearch/SentEval/tree/master/data/probing) for evaluating linguistic properties of your embeddings. 
```
    cd data/downstream/
    ./get_transfer_data.bash
```
* Download pretained Glove embeddings:

```
    mkdir pretrained
    cd pretrained
    wget http://nlp.stanford.edu/data/glove.840B.300d.zip
   
```

* The following code evaluates Glove pretrained embeddings on different NLP downstream tasks.

In [4]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
import SentEval.examples.data as data
import os

# Set PATHs
# path to senteval
PATH_TO_SENTEVAL = 'SentEval'
# path to the NLP datasets 
PATH_TO_DATA = os.path.join('SentEval','data')
# path to glove embeddings
PATH_TO_VEC = os.path.join('SentEval','pretrained','glove.840B.300d.txt')


# import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval


def prepare(params, samples):
    """
    In this example we are going to load Glove, 
    here you will initialize your model.
    remember to add what you model needs into the params dictionary
    """
    _, params.word2id = data.create_dictionary(samples)
    # load glove/word2vec format 
    params.word_vec = data.get_wordvec(PATH_TO_VEC, params.word2id)
    # dimensionality of glove embeddings
    params.wvec_dim = 300
    return

def batcher(params, batch):
    """
    In this example we use the average of word embeddings as a sentence representation.
    Each batch consists of one vector for sentence.
    Here you can process each sentence of the batch, 
    or a complete batch (you may need masking for that).
    
    """
    # if a sentence is empty dot is set to be the only token
    # you can change it into NULL dependening in your model
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []
    
    for sent in batch:
        sentvec = []
        # the format of a sentence is a lists of words (tokenized and lowercased)
        for word in sent:
            if word in params.word_vec:
                # [number of words, embedding dimensionality]
                sentvec.append(params.word_vec[word])
        if not sentvec:
            vec = np.zeros(params.wvec_dim)
            # [number of words, embedding dimensionality]
            sentvec.append(vec)
        # average of word embeddings for sentence representation
        # [embedding dimansionality]
        sentvec = np.mean(sentvec, 0)
        embeddings.append(sentvec)
    # [batch size, embedding dimensionality]
    embeddings = np.vstack(embeddings)
    return embeddings


# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10}
# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    transfer_tasks = ['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC',
                      'MRPC', 'SICKEntailment', 'STS14']
    # senteval prints the results and returns a dictionary with the scores
    results = se.eval(transfer_tasks)
    print(results)

2018-05-21 14:27:26,594 : ***** Transfer task : MR *****


2018-05-21 14:27:42,415 : Found 723 words with word vectors, out of         735 words
2018-05-21 14:27:42,416 : Generating sentence embeddings
2018-05-21 14:27:42,427 : Generated sentence embeddings
2018-05-21 14:27:42,427 : Training sklearn-LogReg with (inner) 10-fold cross-validation
2018-05-21 14:27:42,565 : Best param found at split 1: l2reg = 1                 with score 64.1
2018-05-21 14:27:42,696 : Best param found at split 2: l2reg = 0.5                 with score 64.0
2018-05-21 14:27:42,803 : Best param found at split 3: l2reg = 4                 with score 67.43
2018-05-21 14:27:42,915 : Best param found at split 4: l2reg = 1                 with score 70.62
2018-05-21 14:27:43,032 : Best param found at split 5: l2reg = 1                 with score 65.48
2018-05-21 14:27:43,155 : Best param found at split 6: l2reg = 4                 with score 66.43
2018-05-21 14:27:43,264 : Best param found at split 7: l2reg = 8  

2018-05-21 14:47:48,390 : Computing embedding for train
2018-05-21 14:47:48,710 : Computed train embeddings
2018-05-21 14:47:48,710 : Computing embedding for dev
2018-05-21 14:47:48,742 : Computed dev embeddings
2018-05-21 14:47:48,742 : Computing embedding for test
2018-05-21 14:47:49,093 : Computed test embeddings
2018-05-21 14:47:49,152 : Training sklearn-LogReg with standard validation..
2018-05-21 14:48:00,142 : [('reg:0.25', 77.8), ('reg:0.5', 80.0), ('reg:1', 80.8), ('reg:2', 81.0), ('reg:4', 80.0), ('reg:8', 80.0)]
2018-05-21 14:48:00,142 : Validation : best param found is reg = 2 with score             81.0
2018-05-21 14:48:00,142 : Evaluating...
2018-05-21 14:48:02,153 : 
Dev acc : 81.0 Test acc : 79.01 for                        SICK entailment

2018-05-21 14:48:02,153 : ***** Transfer task : STS14 *****




FileNotFoundError: [Errno 2] No such file or directory: 'SentEval\\data/downstream/STS/STS14-en-test/STS.input.deft-forum.txt'